# VacationPy
---

## Dependencies and Setup

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [9]:
# Loads the CSV file created in WeatherPy into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Displays sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao tome,0.3365,6.7273,26.45,81,95,2.10,ST,1698523623
1,1,mount pearl,47.5166,-52.7813,13.41,95,99,1.34,CA,1698523900
2,2,pacific grove,36.6177,-121.9166,21.42,62,0,5.81,US,1698523901
3,3,bernardo de irigoyen,-26.2552,-53.6458,19.06,98,99,2.48,AR,1698523901
4,4,grytviken,-54.2811,-36.5092,-0.43,97,100,1.11,GS,1698523625


---

### This section creates a map that displays a point for every city in the `city_data_df` DataFrame

In [10]:
%%capture --no-display

# Configure the map plot
humid_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 1000,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
humid_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### This sections narrows down the `city_data_df` DataFrame based on my ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 19) & (city_data_df["Humidity"] < 75) & (city_data_df["Cloudiness"] < 10)]

# Drop rows with null values
ideal_weather = ideal_weather.dropna(how='any',axis=0) 
ideal_weather

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,pacific grove,36.6177,-121.9166,21.42,62,0,5.81,US,1698523901
44,44,fort bragg,35.1390,-79.0060,30.36,49,0,0.89,US,1698523442
48,48,guerrero negro,27.9769,-114.0611,21.88,60,0,8.15,MX,1698523313
49,49,lihue,21.9789,-159.3672,27.58,66,0,2.57,US,1698523917
83,83,conceicao do araguaia,-8.2578,-49.2647,36.79,29,8,2.59,BR,1698523627
95,95,dosso,13.0490,3.1937,30.11,23,7,1.89,NE,1698523938
122,122,cabo san lucas,22.8909,-109.9124,28.61,57,0,5.66,MX,1698523953
128,128,taoudenni,22.6783,-3.9836,34.04,10,0,3.39,ML,1698523956
164,164,westport,41.1415,-73.3579,28.30,62,6,1.34,US,1698523783
168,168,tindouf,27.6711,-8.1474,25.25,36,0,5.14,DZ,1698523971


### This sections creates a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# An empty column, "Hotel Name," is added to the DataFrame to store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,pacific grove,US,36.6177,-121.9166,62,
44,fort bragg,US,35.1390,-79.0060,49,
48,guerrero negro,MX,27.9769,-114.0611,60,
49,lihue,US,21.9789,-159.3672,66,
83,conceicao do araguaia,BR,-8.2578,-49.2647,29,


### This section uses the Geoapify API to find the first hotel located within 10,000 metres of the coordinates for each city

In [13]:
# Sets parameters to search for a hotel

params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Prints a message to follow up the hotel search
print("Starting hotel search")

# Iterates through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Adds filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},10000"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Sets base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Makes an API request using the params dictionaty
    hotel_request = requests.get(base_url, params=params)
    
    # Converts the API response to JSON format
    name_address = hotel_request.json()
    
    # Grabs the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Logs the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pacific grove - nearest hotel: Pacific Grove Inn
fort bragg - nearest hotel: Airborne Inn Lodging
guerrero negro - nearest hotel: Plaza sal paraiso
lihue - nearest hotel: Kauai Palms
conceicao do araguaia - nearest hotel: No hotel found
dosso - nearest hotel: Galaxy Hotel
cabo san lucas - nearest hotel: Comfort Rooms
taoudenni - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
tindouf - nearest hotel: محمد بوسبي
naze - nearest hotel: ビジネス旅館畠山
mazabuka - nearest hotel: No hotel found
oasis - nearest hotel: No hotel found
minab - nearest hotel: هتل صدف
portel - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
cruz del eje - nearest hotel: Hotel España
kill devil hills - nearest hotel: Mariner Days Inn & Suites
kapa'a - nearest hotel: Pono Kai Resort
ormara - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
goniri - nearest hotel: No hotel found
dublin - nearest hotel: Holiday Inn Dub

,City,Country,Lat,Lng,Humidity,Hotel Name
2,pacific grove,US,36.6177,-121.9166,62,Pacific Grove Inn
44,fort bragg,US,35.1390,-79.0060,49,Airborne Inn Lodging
48,guerrero negro,MX,27.9769,-114.0611,60,Plaza sal paraiso
49,lihue,US,21.9789,-159.3672,66,Kauai Palms
83,conceicao do araguaia,BR,-8.2578,-49.2647,29,No hotel found
95,dosso,NE,13.0490,3.1937,23,Galaxy Hotel
122,cabo san lucas,MX,22.8909,-109.9124,57,Comfort Rooms
128,taoudenni,ML,22.6783,-3.9836,10,No hotel found
164,westport,US,41.1415,-73.3579,62,Norwalk Inn and Conference Center
168,tindouf,DZ,27.6711,-8.1474,36,محمد بوسبي


### This section adds the hotel name and the country as additional information in the hover message for each city in the map

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 1000,
    scale = 1.0,
    color = "City",
    hover_cols = ["Country", "Hotel Name", "Humidity"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)